In [62]:
import requests
import re
from bs4 import BeautifulSoup

In [ ]:
root = "http://gyakorikerdesek.hu"
page = requests.get(root)
soup = BeautifulSoup(page.content, 'html.parser')
html = list(soup.children)[2]

menus = [] 
for menu in soup.find_all(class_='menu'):
    menus.append(menu.a.get("href"))
    
page = requests.get(root + menus[1])
allatok = BeautifulSoup(page.content, 'html.parser')

In [67]:
valasz_hasznos = re.compile("A válasz (\d+)%-ban hasznosnak tűnik.")
iro_hasznos = re.compile("A válaszíró (\d+)%-ban hasznos válaszokat ad.")

k = allatok.find_all(class_="kerdes_lista")[1].find_all("tr")[0].find("a").get("href")

subpage = requests.get(root + "/allatok__allatvedelem__10072086-szerintetek-nincs-tulzasba-vive-manapsag-ez-a-kutyabarat-tema")
print(k)
p = BeautifulSoup(subpage.content, 'html.parser').find_all("td", class_="valaszok vtop")
if p:
    for val in p:
        val_text = val.get_text().strip("\n")
        if val_text:
            print(val_text)
            m = valasz_hasznos.search(val_text)
            print(m)

/allatok__egyeb-kerdesek__10073226-milyen-allatot-lathattam
Nincs.A válasz 86%-ban hasznosnak tűnik. A válaszíró 75%-ban hasznos válaszokat ad.
<_sre.SRE_Match object; span=(6, 40), match='A válasz 86%-ban hasznosnak tűnik.'>
Nincs.Nekem meg az olyan unatkozó mindenbe belekötő savanyu emberekbol van elegem mint te. Tan ki kellene téged meg a hozzad hasonlókat tiltani az üzletekből, nem? Sőt, tudok jobbat. Kilovunk a Holdra, okes?A válasz 74%-ban hasznosnak tűnik. A válaszíró 73%-ban hasznos válaszokat ad.
<_sre.SRE_Match object; span=(207, 241), match='A válasz 74%-ban hasznosnak tűnik.'>
A kutyát legalább lehet szeretni...nem úgy, mint például téged.A válasz 73%-ban hasznosnak tűnik. A válaszíró 66%-ban hasznos válaszokat ad.
<_sre.SRE_Match object; span=(63, 97), match='A válasz 73%-ban hasznosnak tűnik.'>
Nincs, eddig is ugyanez volt, max. te csak most vetted észre. És nem, nem értek egyet az i.dióta elméleteddel. Én allergiás vagyok a kutyaszőrre, de abszolút nem zavar egyáltalán, 

In [ ]:
corp = {}
for i,kerdes in enumerate(allatok.find_all(class_="kerdes_lista")[1].find_all("tr")):
    kerdes_url = kerdes.find("a").get("href")
    subpage = requests.get(root + kerdes_url)
    allatok_kerdes = BeautifulSoup(subpage.content, 'html.parser')
    hosszu_kerdes = allatok_kerdes.find(class_="kerdes").find_all("td")[2].get_text()
    rovid_kerdes = allatok_kerdes.find(class_="kerdes").find_all("td")[2].h1.get_text()
    kategoriak = []
    for kategoria in allatok_kerdes.find("select").find_all("option")[1:]:
        kategoriak.append(kategoria.get_text())
    corp[i] = {"rovid_kerdes": rovid_kerdes, "hosszu_kerdes": hosszu_kerdes, "kategoriak": kategoriak}
    print(i)

In [ ]:
for k in range(1,1000):
    page = requests.get(root + "/allatok__oldal-"+str(k))
    allatok = BeautifulSoup(page.content, 'html.parser')
    kerdesek_raw = allatok.find_all(class_="kerdes_lista")
    if len(kerdesek_raw) == 1:
        kerdesek = kerdesek_raw[0].find_all("tr")
    elif len(kerdesek_raw) == 2:
        kerdesek = kerdesek_raw[1].find_all("tr")
    for kerdes in kerdesek:
        kerdes_url = kerdes.find("a").get("href")
        subpage = requests.get(root + kerdes_url)
        allatok_kerdes = BeautifulSoup(subpage.content, 'html.parser')
        hosszu_kerdes = allatok_kerdes.find(class_="kerdes").find_all("td")[2].get_text()
        rovid_kerdes = allatok_kerdes.find(class_="kerdes").find_all("td")[2].h1.get_text()
        kategoriak = []
        for kategoria in allatok_kerdes.find("select").find_all("option")[1:]:
            kategoriak.append(kategoria.get_text())
        i += 1
        corp[i] = {"rovid_kerdes": rovid_kerdes, "hosszu_kerdes": hosszu_kerdes, "kategoriak": kategoriak}
        print(i)
    print("page: " + str(k))

In [ ]:
page = requests.get(root + "/allatok__kutyak__9989071-meg-nem-volt-kutyam-airdale-terrier-lakasban-tartasarol-mit-gondoltok")

In [ ]:
allatok_kerdes = BeautifulSoup(page.content, 'html.parser')

In [ ]:
allatok_kerdes

In [ ]:
allatok_kerdes.find(class_="kerdes").find_all("td")[2].get_text()

In [ ]:
allatok_kerdes.find(class_="kerdes").find_all("td")[2].h1.get_text()

In [ ]:
allatok_kerdes.find("select").find("option").get_text()

In [ ]:
import json
with open("gyakori_corpus", "w+") as f:
    for ind in corp:
        kategoriak = corp[ind]["kategoriak"]
        rovid_kerdes = corp[ind]["rovid_kerdes"]
        hosszu_kerdes = ' '.join(corp[ind]["hosszu_kerdes"].split("\n")[1:])
        f.write(",".join(kategoriak) + "\t" + rovid_kerdes + "\t" + hosszu_kerdes + "\n")
        #f.write(json.dumps(corp[ind]))

In [ ]:
kerdes_url

In [ ]:
corp

In [ ]:
filtered_corp

In [ ]:
inds = []
for c in corp:
    hosszu_kerdes = corp[c]["hosszu_kerdes"].strip("\n").replace("(adsbygoogle = window.adsbygoogle || []).push({});","").split("\n")
    hosszu_kerdes = " ".join(hosszu_kerdes)
    if "Keress kérdéseket hasonló témákban:" in hosszu_kerdes:
        kerdes = hosszu_kerdes.split("Keress kérdéseket hasonló témákban:")
        corp[c]["hosszu_kerdes"] = kerdes[0].strip().replace("\n"," ")
        corp[c]["keywords"] = [i.strip() for i in kerdes[1].split(",")]
        kat = corp[c]["kategoriak"]
        kat_filt = [i.replace("» ", "") for i in kat]
        corp[c]["kategoriak"] = kat_filt
        inds.append(c)
        
filtered_corp = [corp[i] for i in corp if i in inds]

In [ ]:
import json
with open("gyakori_corpus", "w+") as f:
    for ind in inds:
        kategoriak = corp[ind]["kategoriak"]
        rovid_kerdes = corp[ind]["rovid_kerdes"]
        hosszu_kerdes = ' '.join(corp[ind]["hosszu_kerdes"].split("\n")[1:])
        f.write(",".join(kategoriak) + "\t" + rovid_kerdes + "\t" + hosszu_kerdes + "\n")